In [28]:
import pandas as pd
from pathlib import Path
import numpy as np
import scipy.stats as stats
import requests
from bs4 import BeautifulSoup
from langdetect import detect, DetectorFactory

In [18]:
#genius api
genius_api_key = 'gQVkGL-wnevLS3t39UOjl1a9x3smebFPcMX6awdjS-61kw-NrQEQmWWDbNXzUWjz'
genius_url = 'https://api.genius.com'

In [19]:
spotify_songs_csv = Path("Resources/spotify_top_songs.csv")
spotify_songs_df = pd.read_csv(spotify_songs_csv)
spotify_songs_df.drop (columns=["Top 10 (xTimes)", 'Peak Position', 'Peak Position (xTimes)', 'Peak Streams', ], inplace=True)

spotify_songs_df.head()

,Position,Artist Name,Song Name,Days,Total Streams
0,1,Post Malone,Sunflower SpiderMan: Into the SpiderVerse,1506,883369738
1,2,Juice WRLD,Lucid Dreams,1673,864832399
2,3,Lil Uzi Vert,XO TOUR Llif3,1853,781153024
3,4,J. Cole,No Role Modelz,2547,734857487
4,5,Post Malone,rockstar,1223,718865961


In [20]:
spotify_songs_df['Lyrics URL'] = ''
spotify_songs_df['Lyrics'] = ''
spotify_songs_df['Language'] = ''
spotify_songs_df.head()

,Position,Artist Name,Song Name,Days,Total Streams,Lyrics URL,Lyrics,Language
0,1,Post Malone,Sunflower SpiderMan: Into the SpiderVerse,1506,883369738,,,
1,2,Juice WRLD,Lucid Dreams,1673,864832399,,,
2,3,Lil Uzi Vert,XO TOUR Llif3,1853,781153024,,,
3,4,J. Cole,No Role Modelz,2547,734857487,,,
4,5,Post Malone,rockstar,1223,718865961,,,


In [21]:
top_100_df = spotify_songs_df.sort_values(by='Total Streams', ascending=False).head(100)
top_100_df

,Position,Artist Name,Song Name,Days,Total Streams,Lyrics URL,Lyrics,Language
0,1,Post Malone,Sunflower SpiderMan: Into the SpiderVerse,1506,883369738,,,
1,2,Juice WRLD,Lucid Dreams,1673,864832399,,,
2,3,Lil Uzi Vert,XO TOUR Llif3,1853,781153024,,,
3,4,J. Cole,No Role Modelz,2547,734857487,,,
4,5,Post Malone,rockstar,1223,718865961,,,
...,...,...,...,...,...,...,...,...
95,96,Travis Scott,BUTTERFLY EFFECT,682,276763296,,,
96,97,Brenda Lee,Rockin' Around The Christmas Tree,340,276529912,,,
97,98,Drake,Knife Talk,471,275693546,,,
98,99,Pop Smoke,For The Night,512,274720487,,,


In [ ]:
def search_genius(row):
    base_url = "https://api.genius.com/search"
    token = "gQVkGL-wnevLS3t39UOjl1a9x3smebFPcMX6awdjS-61kw-NrQEQmWWDbNXzUWjz"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "q": f"{row['Song Name']} {row['Artist Name']}"
    }
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        
        hits = data.get("response", {}).get("hits", [])
        if hits:
            return hits[0].get("result", {}).get("url", "No URL Found")
        else:
            return "No Results"
    except Exception as e:
        return f"Error: {e}"


top_100_df['Lyrics URL'] = top_100_df.apply(search_genius, axis=1)
top_100_df.head()

,Position,Artist Name,Song Name,Days,Total Streams,Lyrics URL,Lyrics,Language
0,1,Post Malone,Sunflower SpiderMan: Into the SpiderVerse,1506,883369738,https://genius.com/Apple-music-post-malone-ess...,,
1,2,Juice WRLD,Lucid Dreams,1673,864832399,https://genius.com/Juice-wrld-lucid-dreams-lyrics,,
2,3,Lil Uzi Vert,XO TOUR Llif3,1853,781153024,https://genius.com/Lil-uzi-vert-xo-tour-llif3-...,,
3,4,J. Cole,No Role Modelz,2547,734857487,https://genius.com/J-cole-no-role-modelz-lyrics,,
4,5,Post Malone,rockstar,1223,718865961,https://genius.com/Post-malone-rockstar-lyrics,,


In [26]:
def scrape_lyrics(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the lyrics container
        lyrics_container = soup.find('div', class_='Lyrics__Container-sc-1ynbvzw-1 kUgSbL')  # Class name might change; inspect page
        if lyrics_container:
            return lyrics_container.get_text(separator="\n").strip()
        return "Lyrics not found"
    except Exception as e:
        return f"Error: {e}"

# Apply scraping function
top_100_df['Lyrics'] = top_100_df['Lyrics URL'].apply(scrape_lyrics)

In [27]:
top_100_df.head()

,Position,Artist Name,Song Name,Days,Total Streams,Lyrics URL,Lyrics,Language
0,1,Post Malone,Sunflower SpiderMan: Into the SpiderVerse,1506,883369738,https://genius.com/Apple-music-post-malone-ess...,1. Post Malone- Circles\n2. Post Malone- Wow\n...,
1,2,Juice WRLD,Lucid Dreams,1673,864832399,https://genius.com/Juice-wrld-lucid-dreams-lyrics,"[Intro]\nEnviyon on the mix\nNo, no, no, no\nN...",
2,3,Lil Uzi Vert,XO TOUR Llif3,1853,781153024,https://genius.com/Lil-uzi-vert-xo-tour-llif3-...,"[Intro]\nAre you alright?\nI'm alright, I'm qu...",
3,4,J. Cole,No Role Modelz,2547,734857487,https://genius.com/J-cole-no-role-modelz-lyrics,"[Verse 1]\nFirst things first: rest in peace, ...",
4,5,Post Malone,rockstar,1223,718865961,https://genius.com/Post-malone-rockstar-lyrics,[Intro: Post Malone]\nHahahahaha\nTank God\nAy...,


In [29]:
DetectorFactory.seed = 0

def detect_language(lyrics):
    try:
        return detect(lyrics)
    except Exception as e:
        return f"Error: {e}"

# Apply the function to detect language
top_100_df['Language'] = top_100_df['Lyrics'].apply(detect_language)
top_100_df.head()

,Position,Artist Name,Song Name,Days,Total Streams,Lyrics URL,Lyrics,Language
0,1,Post Malone,Sunflower SpiderMan: Into the SpiderVerse,1506,883369738,https://genius.com/Apple-music-post-malone-ess...,1. Post Malone- Circles\n2. Post Malone- Wow\n...,en
1,2,Juice WRLD,Lucid Dreams,1673,864832399,https://genius.com/Juice-wrld-lucid-dreams-lyrics,"[Intro]\nEnviyon on the mix\nNo, no, no, no\nN...",en
2,3,Lil Uzi Vert,XO TOUR Llif3,1853,781153024,https://genius.com/Lil-uzi-vert-xo-tour-llif3-...,"[Intro]\nAre you alright?\nI'm alright, I'm qu...",en
3,4,J. Cole,No Role Modelz,2547,734857487,https://genius.com/J-cole-no-role-modelz-lyrics,"[Verse 1]\nFirst things first: rest in peace, ...",en
4,5,Post Malone,rockstar,1223,718865961,https://genius.com/Post-malone-rockstar-lyrics,[Intro: Post Malone]\nHahahahaha\nTank God\nAy...,en


In [30]:
unique_language = top_100_df['Language'].unique()
print(unique_language)

['en']
